In [1]:
import pandas as pd
import numpy as np
from ipynb.fs.defs.add_features import add_features
import os
import pandas_datareader as pdr
import time
import chart_studio.plotly as py
import plotly.graph_objects as go
from plotly import tools
from datetime import datetime, timedelta

# Time Series / ARIMA
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import autocorrelation_plot
from sklearn import metrics
from datetime import datetime, timedelta
from pmdarima import auto_arima

In [2]:
stock_list = ['RACE', 'COST', 'WMT', 'AMZN', 'BA']

In [3]:
def ideal_arima(stock_close, max_p=20, max_q=20):
    """Function to run auto_arima for ideal parameters
    Parameters:
    stock_close(dataframe): df of the stock close price
    max_p(int): max number of p
    max_q(int): max number of q
    
    Returns the ideal ARIMA parameters
    """
    
    # Run auto ARIMA based on the parameteres below
    stepwise_model = auto_arima(stock_close, start_p=1, start_q=1,
                           max_p=max_p, max_q=max_q, m=12, start_P=0, seasonal=True,
                           d=None, max_d=1, D=1, error_action='ignore',  
                           suppress_warnings=True, stepwise=True)
    
    # Append ideal auto ARIMA parameters to list
    tmp = stepwise_model.get_params('order')
    order = tmp.get('order')
    
    return order

In [4]:
def forecast_arima(stock_close, n_days=10):
    """Run ideal ARIMA function and forecast closing stock price 
    
    Parameters:
    stock_close(dataframe): df of the stock close price
    n_days(int): number of days to forecast
    """
    
    # Create list of all stock close values
    df1 = stock_close
    X = df1.values
    history = [x for x in X]
    
    # Create forecast prediction list
    predictions = list()
    #order = ideal_arima(stock_close)
    #print(order)
    
    # Run ARIMA for each day to forecast
    for t in range(n_days):
        model = ARIMA(history, order=(0,1,0))
        model_fit = model.fit()
        output = model_fit.forecast()
        
        # Append forecast to predictions and list of stock values
        yhat = output[0]
        predictions.append(yhat)
        history.append(yhat)
        
    # Get last date from dataframe
    last_date = df1.index[-1] 
    
    # create date index for forecast
    date_index = []
    for x in range(1, n_days+1):
        x = date_by_adding_business_days(last_date, x)
        date_index.append(x)
        
    # Create Dataframe with predictions
    df_pred = pd.DataFrame(predictions)

    # Add prediction dates to index
    df_pred.index = date_index
    df_pred.rename(columns={0:'pred_close'}, inplace=True)
    df_forecast = pd.concat([stock_close, df_pred], axis=1)
    
    return df_forecast

def date_by_adding_business_days(from_date, add_days):
    business_days_to_add = add_days
    current_date = from_date
    while business_days_to_add > 0:
        current_date += timedelta(days=1)
        weekday = current_date.weekday()
        if weekday >= 5: # sunday = 6
            continue
        business_days_to_add -= 1
    return current_date

In [5]:
def date_by_adding_business_days(last_date, add_days):
    """Adds and returns the business date based on parameters
    
    Parameters:
    last_date(date): current date
    add_days(int): number of days to add to last date
    """

    business_days_to_add = add_days
    current_date = last_date
    while business_days_to_add > 0:
        current_date += timedelta(days=1)
        weekday = current_date.weekday()
        if weekday >= 5: # sunday = 6
            continue
        business_days_to_add -= 1
    
    return current_date

In [6]:
def forecast_chart(stock, api, n_days):
    """ Returns generated chart of historical prices with the forecasted price

    Parameters:
    stock(str): stock ticker
    api(str): api for Tiingo
    n_days(int): Number of days in the forecast
    """
    
    df = pdr.get_data_tiingo(stock, api_key=api)
    df = df.tail(100)
    df.reset_index(inplace=True)
    df.index = df['date']
    df_forecast = forecast_arima(df['close'], n_days)
    
    # Historical prices and forecasted prices
    trace0 = go.Scatter(x=df_forecast.index, y=df_forecast['close'],
                        mode='lines', name='Historical', 
                        line=dict(color='blue', width=1), yaxis='y')

    trace1 = go.Scatter(x=df_forecast.index, y=df_forecast['pred_close'],
                        mode='lines', name='Prediction', 
                        line=dict(color='red', width=1), yaxis='y')
    
    data = [trace0, trace1]
    
    # Additional layout and format of the chart
    layout = go.Layout(
        yaxis=dict(domain=[0, 1], title='Price (USD)'),

        title='{} Stock Forecast Chart'.format(stock),

        xaxis=dict(
            title='Date',
            
            # Add range selector buttons to the chart
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=3,
                         label='3m',
                         step='month',
                         stepmode='backward'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(
                visible = False
            ),
            type='date'
        )
    )
    
    # Generate chart based on data and layout features
    fig = go.Figure(data=data, layout=layout)
    
    return fig.show()

In [7]:
def roi_calc(stock_list, api, n_days=10):
    """Iterates through stock list and calculates ROI based
    on the ARIMA forecast.
    
    Parameters:
    stock_list(list): list of stocks to iterate
    api(str): api for Tiingo
    n_days(int): number of days to forecast
    
    Returns list of top three stocks, forecast, and predicted ROI
    """

    start_time = time.time()
    roi = {}
    
    for x in stock_list:
        # Create dataframe for each ticker
        df = pdr.get_data_tiingo(x, api_key=api)
        df = df.tail(1000)
        df.reset_index(inplace=True)
        df.index = df['date']
        stock_close = df['close']
        
        # Get the forecast for each ticker
        df_forecast = forecast_arima(stock_close, n_days)
        
        # Calculate ROI
        last_close = df['close'][-1]
        forecast = df_forecast['pred_close'][-1]
        calc_roi = ((forecast - last_close) / last_close) * 100
        
        # Append to forecast and ROI dictionaries
        roi.update({x:calc_roi})

    # Print ROI rates for top 3
    best_roi = sorted(roi, key=roi.get, reverse=True)[:3]  
    print('Top 3 Forecasted ROI:')
    
    for x in best_roi:
        print('{}: {}%'.format(x, roi[x]))
        forecast_chart(x, api, n_days)
    
    print("\n--- %s seconds ---" % (time.time() - start_time))  
    return roi